In [ ]:
#! Python3
# by Jacob Kovach
# Confidential and Proprietary

import numpy as np, pandas as pd, urllib.request, spacy, re, os
from tqdm import tqdm
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from nltk import tokenize
from pandas_datareader import DataReader
from collections import Counter

nlp = spacy.load('en')
tqdm.pandas()

earnings_df = pd.read_csv('/Users/jkovach/Downloads/earnings-call-transcripts/_raw_data.csv')
earnings_df = earnings_df.drop('Unnamed: 0', 1)

def clean_and_tokenize(ts):
    ts = re.sub(r'\d+', '', ts)
    ts_doc = nlp(ts)
    return ts_doc
    #content = re.sub(r'[A-Z]+', '', content)

earnings_df['Tokens'] = earnings_df['transcript'].progress_apply(clean_and_tokenize)

earnings_df.head()

# Create single text file and run NLP tokenizer

/Users/jkovach/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
 78%|███████▊  | 683/875 [17:21<03:57,  1.24s/it]

In [ ]:
# Utility function to create a list of the 200 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(200)]

# get set amount of words
common_words = []
for i in range(earnings_df.shape[0]):
    cw = bag_of_words(earnings_df.loc[i, 'Tokens'])
    common_words = set(list(common_words) + cw)
    
print(len(common_words))
print(common_words)

In [ ]:
# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['Tokens'] = sentences['Tokens']
    df['Movement'] = sentences['Movement']
    df.loc[:, common_words] = 0
    df = df.reset_index(drop=True)

    # Process each row, counting the occurrence of words in each sentence.
    for i, transcript in enumerate(df['Tokens']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in transcript
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        #print(words)
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))

    return df


word_counts = bow_features(earnings_df, common_words)
word_counts.head()

# Word to vec, get 

In [ ]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


rfc = ensemble.RandomForestClassifier()
y = word_counts['Movement']
X = np.array(word_counts.drop(['Tokens','Movement'], 1))

print("Class Balance: {}".format(sorted(Counter(y).items())))

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))
confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression() 
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

In [ ]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))